# Example making grid search for Tab Net

Author: Hartorn

File from [github.com](https://github.com/dreamquark-ai/tabnet/issues/80#issuecomment-623954898).

> Note: In this notebook, I am studying the article mentioned above. Some changes may have been made to the code during its implementation.

In [ ]:
!pip install requests black nb_black
%load_ext nb_black

In [1]:
import os
from pathlib import Path

from requests import get
import pandas as pd
import numpy as np
import torch

np.random.seed(0)

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier

# Utilities

In [2]:
def download(url, out, force=False, verify=True):
    out.parent.mkdir(parents=True, exist_ok=True)
    if force:
        print(f"Removing file at {str(out)}")
        out.unlink()

    if out.exists():
        print("File already exists.")
        return
    print(f"Downloading {url} at {str(out)} ...")
    # open in binary mode
    with out.open(mode="wb") as file:
        # get request
        response = get(url, verify=verify)
        for chunk in response.iter_content(100000):
            # write to file
            file.write(chunk)

In [3]:
UNKNOWN_VALUE = ["Unkn0wnV@lue"]


class SafeLabelEncoder(LabelEncoder):
    """
    Safe label encoder, encoding every unknown value as Unkn0wnV@lue.
    """

    def fit(self, y):
        """
        Fit the label encoder, by casting the numpy array as a string, then adding the code for unknown.
        
        Parameters
        ----------
        y : numpy array
            the values to fit
        
        Returns
        -------
        SafeLabelEncoder
            itself, fitted
        """
        return super().fit(np.concatenate((y.astype("str"), UNKNOWN_VALUE)))

    def fit_transform(self, y):
        """
        Fit the encoder, then transform the input data and returns it.
        
        Parameters
        ----------
        y : numpy array
            the values to fit
        
        Returns
        -------
        numpy array
            the encoded data
        """
        self.fit(y)
        return super().transform(y)

    def transform(self, y):
        """
        Transform the input data and returns it.
        
        Parameters
        ----------
        y : numpy array
            the values to fit
        
        Returns
        -------
        numpy array
            the encoded data
        """
        return super().transform(
            np.where(
                np.isin(y.astype("str"), self.classes_), y.astype("str"), UNKNOWN_VALUE
            )
        )



# Download census-income dataset

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
url_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

dataset_name = "census-income"
out = Path(os.getcwd() + "/datasets/" + dataset_name + ".csv")
out_test = Path(os.getcwd() + "/datasets/" + dataset_name + "_test.csv")

download(url, out, force=False)
download(url_test, out_test, force=False)

File already exists.
File already exists.


# Load data and split

In [5]:
cols = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "target",
]

In [6]:
train = pd.read_csv(out, names=cols)
test = pd.read_csv(out_test, names=cols, skiprows=2)
target = "target"

train[target] = train[target].str.strip()
# Test has . in label, let's clean it
test[target] = test[target].str.strip().str.strip(".")

In [7]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  target          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
2,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
3,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
4,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16280 entries, 0 to 16279
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16280 non-null  int64 
 1   workclass       16280 non-null  object
 2   fnlwgt          16280 non-null  int64 
 3   education       16280 non-null  object
 4   education-num   16280 non-null  int64 
 5   marital-status  16280 non-null  object
 6   occupation      16280 non-null  object
 7   relationship    16280 non-null  object
 8   race            16280 non-null  object
 9   sex             16280 non-null  object
 10  capital-gain    16280 non-null  int64 
 11  capital-loss    16280 non-null  int64 
 12  hours-per-week  16280 non-null  int64 
 13  native-country  16280 non-null  object
 14  target          16280 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


In [11]:
if "Set" not in train.columns:
    train["Set"] = np.random.choice(
        ["train", "valid"], p=[0.8, 0.2], size=(train.shape[0],)
    )

train_indices = train[train.Set == "train"].index
valid_indices = train[train.Set == "valid"].index

In [12]:
used_columns = list(set(train.columns.tolist()) - set([target]) - set(["Set"]))
used_columns

['capital-gain',
 'native-country',
 'capital-loss',
 'relationship',
 'education-num',
 'hours-per-week',
 'education',
 'age',
 'sex',
 'workclass',
 'fnlwgt',
 'race',
 'occupation',
 'marital-status']

# Simple preprocessing

Label encode categorical features and fill empty cells.

In [13]:
nunique = train[used_columns].nunique()
types = train[used_columns].dtypes

cat_cols = train[used_columns].columns[(nunique < 200) | (types == "object")]
other_cols = train[used_columns].columns[~train[used_columns].columns.isin(cat_cols)]
print(cat_cols)
print(other_cols)

Index(['capital-gain', 'native-country', 'capital-loss', 'relationship',
       'education-num', 'hours-per-week', 'education', 'age', 'sex',
       'workclass', 'race', 'occupation', 'marital-status'],
      dtype='object')
Index(['fnlwgt'], dtype='object')


In [14]:
nunique["education"]

16

In [15]:
# Fillna
train[cat_cols] = train[cat_cols].astype("str")
train[other_cols] = train[other_cols].fillna(train[other_cols].mean())

test[cat_cols] = test[cat_cols].astype("str")
test[other_cols] = test[other_cols].fillna(train[other_cols].mean())

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  object
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  object
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  object
 11  capital-loss    32561 non-null  object
 12  hours-per-week  32561 non-null  object
 13  native-country  32561 non-null  object
 14  target          32561 non-null  object
 15  Set             32561 non-null  object
dtypes: int64(1), object(15)
memory usage: 4.0+ MB


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16280 entries, 0 to 16279
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             16280 non-null  object
 1   workclass       16280 non-null  object
 2   fnlwgt          16280 non-null  int64 
 3   education       16280 non-null  object
 4   education-num   16280 non-null  object
 5   marital-status  16280 non-null  object
 6   occupation      16280 non-null  object
 7   relationship    16280 non-null  object
 8   race            16280 non-null  object
 9   sex             16280 non-null  object
 10  capital-gain    16280 non-null  object
 11  capital-loss    16280 non-null  object
 12  hours-per-week  16280 non-null  object
 13  native-country  16280 non-null  object
 14  target          16280 non-null  object
dtypes: int64(1), object(14)
memory usage: 1.9+ MB


In [18]:
train.isnull().sum().sum()

0

In [19]:
enc = {}
for col in cat_cols:
    label_enc = SafeLabelEncoder()
    enc[col] = label_enc
    train[col] = label_enc.fit_transform(train[col])
    test[col] = label_enc.transform(test[col])
enc[target] = SafeLabelEncoder()
train[target] = enc[target].fit_transform(train[target])
test[target] = enc[target].transform(test[target])

enc

{'capital-gain': SafeLabelEncoder(),
 'native-country': SafeLabelEncoder(),
 'capital-loss': SafeLabelEncoder(),
 'relationship': SafeLabelEncoder(),
 'education-num': SafeLabelEncoder(),
 'hours-per-week': SafeLabelEncoder(),
 'education': SafeLabelEncoder(),
 'age': SafeLabelEncoder(),
 'sex': SafeLabelEncoder(),
 'workclass': SafeLabelEncoder(),
 'race': SafeLabelEncoder(),
 'occupation': SafeLabelEncoder(),
 'marital-status': SafeLabelEncoder(),
 'target': SafeLabelEncoder()}

# Define categorical features for categorical embeddings

In [20]:
unused_feat = ["Set"]

cat_idxs = [i for i, f in enumerate(used_columns) if f in cat_cols]
cat_dims = [len(enc[f].classes_) for f in used_columns if f in cat_cols]
print(cat_idxs)
print(cat_dims)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13]
[120, 43, 93, 7, 17, 95, 17, 74, 3, 10, 6, 16, 8]


# Training one model

In [21]:
X = train[used_columns].values
y = train[target].values

X_test = test[used_columns].values
y_test = test[target].values

In [24]:
X.dtype

dtype('int64')

In [25]:
X[:,0].dtype

dtype('int64')

In [27]:
type(X[:,0][0])

numpy.int64

In [47]:
for idxs in cat_idxs:
    print('X idx:{} - unique:{}\n\n'.format(idxs, np.unique(X[:,idxs], return_counts=True)))

X idx:0 - unique:(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118]), array([29849,    43,    25,     6,    12,     4,     1,     6,     8,
           2,     3,    27,    41,     7,     3,    26,     1,     7,
           5,   347,    15,     6,     1,     7,     7,     6,     2,
          37,     3,     4,     5,     2,     9,    48,    23,    16,
           1,     5,     5,     6, 

# Random Grid search

In [28]:
num_workers = os.cpu_count() if torch.cuda.is_available() else 0

In [29]:
class TabNetTuner(TabNetClassifier):
    def fit(self, X, y, *args, **kwargs):
        # Dirty trick => would be better to add n_d in grid, or fix it in __init__ of tuner
        self.n_d = self.n_a
        X_train, X_valid, y_train, y_valid = train_test_split(
            X, y, test_size=0.2, random_state=0, shuffle=True, stratify=y
        )
        return super().fit(
            X_train,
            y_train,
            patience=20,
            # X_valid=X_valid,
            # y_valid=y_valid,
            eval_set=[(X_valid, y_valid)],
            num_workers=num_workers,
            max_epochs=1000,
            batch_size=1024,
            virtual_batch_size=128,
        )

In [30]:
clf = TabNetTuner(cat_idxs=cat_idxs, cat_dims=cat_dims, device_name='cpu')

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [31]:
# Let's generate embedding size based on cat dims
cat_emb_dim_list = []
for max_dim in [1, 5, 10, 20, 50]:
    cat_emb_dim_list.append([min(nb // 2, max_dim) for nb in cat_dims])
cat_emb_dim_list

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 3, 5, 4],
 [10, 10, 10, 3, 8, 10, 8, 10, 1, 5, 3, 8, 4],
 [20, 20, 20, 3, 8, 20, 8, 20, 1, 5, 3, 8, 4],
 [50, 21, 46, 3, 8, 47, 8, 37, 1, 5, 3, 8, 4]]

In [32]:
grid = {
    "n_a": [3, 5, 8, 13, 21],
    # "n_d": [8], #
    "cat_emb_dim": cat_emb_dim_list,
    "n_independent": [0, 1, 2, 5],
    "n_shared": [0, 1, 2],
    "n_steps": [1, 3, 5, 8],
    "clip_value": [1],
    "gamma": [0.5, 1.3, 3],
    "momentum": [0.1, 0.05, 0.02, 0.005],
    "lambda_sparse": [0.1, 0.01, 0.001],
    # "lr": [0.1, 0.02, 0.001],
    "optimizer_params": [
        {'lr': 0.01},
        {'lr': 0.02},
        {'lr': 0.001}],
    "verbose": [1]
    # optimizer_fn
}

In [33]:
search = RandomizedSearchCV(
    clf,
    grid,
    n_iter=90,
    scoring="roc_auc",
    n_jobs=1,
    # iid=False,
    refit=False,
    cv=[(train_indices, valid_indices)],
    verbose=1,
    pre_dispatch=0,
    random_state=0,
    return_train_score=False,
)

In [34]:
X = train[used_columns].values
y = train[target].values

In [35]:
X.shape

(32561, 14)

In [36]:
search.fit(X, y)
search.best_params_

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


Fitting 1 folds for each of 90 candidates, totalling 90 fits
epoch 0  | loss: 0.59811 | val_0_auc: 0.70382 |  0:00:02s
epoch 1  | loss: 0.43932 | val_0_auc: 0.78439 |  0:00:03s
epoch 2  | loss: 0.38792 | val_0_auc: 0.82831 |  0:00:04s
epoch 3  | loss: 0.35251 | val_0_auc: 0.84834 |  0:00:05s
epoch 4  | loss: 0.33348 | val_0_auc: 0.86224 |  0:00:06s
epoch 5  | loss: 0.31918 | val_0_auc: 0.86639 |  0:00:07s
epoch 6  | loss: 0.30787 | val_0_auc: 0.87346 |  0:00:07s
epoch 7  | loss: 0.29866 | val_0_auc: 0.87826 |  0:00:08s
epoch 8  | loss: 0.28791 | val_0_auc: 0.87967 |  0:00:09s
epoch 9  | loss: 0.28056 | val_0_auc: 0.8815  |  0:00:10s
epoch 10 | loss: 0.27559 | val_0_auc: 0.88471 |  0:00:11s
epoch 11 | loss: 0.27035 | val_0_auc: 0.8879  |  0:00:12s
epoch 12 | loss: 0.26574 | val_0_auc: 0.89339 |  0:00:13s
epoch 13 | loss: 0.26266 | val_0_auc: 0.89503 |  0:00:14s
epoch 14 | loss: 0.25873 | val_0_auc: 0.89554 |  0:00:15s
epoch 15 | loss: 0.25333 | val_0_auc: 0.89381 |  0:00:16s
epoch 16 | 

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.6916  | val_0_auc: 0.62325 |  0:00:01s
epoch 1  | loss: 0.54812 | val_0_auc: 0.66734 |  0:00:03s
epoch 2  | loss: 0.53206 | val_0_auc: 0.7289  |  0:00:06s
epoch 3  | loss: 0.49039 | val_0_auc: 0.77537 |  0:00:08s
epoch 4  | loss: 0.47283 | val_0_auc: 0.77575 |  0:00:10s
epoch 5  | loss: 0.47535 | val_0_auc: 0.77961 |  0:00:11s
epoch 6  | loss: 0.45526 | val_0_auc: 0.79757 |  0:00:13s
epoch 7  | loss: 0.44773 | val_0_auc: 0.79994 |  0:00:15s
epoch 8  | loss: 0.44362 | val_0_auc: 0.79878 |  0:00:16s
epoch 9  | loss: 0.43448 | val_0_auc: 0.82269 |  0:00:18s
epoch 10 | loss: 0.41718 | val_0_auc: 0.83518 |  0:00:20s
epoch 11 | loss: 0.41083 | val_0_auc: 0.83882 |  0:00:21s
epoch 12 | loss: 0.40075 | val_0_auc: 0.84101 |  0:00:23s
epoch 13 | loss: 0.39254 | val_0_auc: 0.8485  |  0:00:24s
epoch 14 | loss: 0.38992 | val_0_auc: 0.85714 |  0:00:26s
epoch 15 | loss: 0.38083 | val_0_auc: 0.86024 |  0:00:27s
epoch 16 | loss: 0.3799  | val_0_auc: 0.85881 |  0:00:29s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 0.90113 | val_0_auc: 0.63951 |  0:00:01s
epoch 1  | loss: 0.69731 | val_0_auc: 0.77876 |  0:00:03s
epoch 2  | loss: 0.62132 | val_0_auc: 0.7986  |  0:00:05s
epoch 3  | loss: 0.57145 | val_0_auc: 0.82037 |  0:00:07s
epoch 4  | loss: 0.52619 | val_0_auc: 0.82953 |  0:00:09s
epoch 5  | loss: 0.50315 | val_0_auc: 0.83951 |  0:00:11s
epoch 6  | loss: 0.48044 | val_0_auc: 0.84708 |  0:00:13s
epoch 7  | loss: 0.46274 | val_0_auc: 0.85472 |  0:00:15s
epoch 8  | loss: 0.4456  | val_0_auc: 0.85891 |  0:00:17s
epoch 9  | loss: 0.43177 | val_0_auc: 0.86166 |  0:00:19s
epoch 10 | loss: 0.42338 | val_0_auc: 0.86434 |  0:00:21s
epoch 11 | loss: 0.40961 | val_0_auc: 0.86836 |  0:00:23s
epoch 12 | loss: 0.40325 | val_0_auc: 0.87248 |  0:00:25s
epoch 13 | loss: 0.39683 | val_0_auc: 0.87168 |  0:00:27s
epoch 14 | loss: 0.38749 | val_0_auc: 0.87472 |  0:00:29s
epoch 15 | loss: 0.38217 | val_0_auc: 0.87745 |  0:00:31s
epoch 16 | loss: 0.37637 | val_0_auc: 0.87915 |  0:00:33s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.6436  | val_0_auc: 0.53124 |  0:00:00s
epoch 1  | loss: 0.5853  | val_0_auc: 0.54108 |  0:00:01s
epoch 2  | loss: 0.54934 | val_0_auc: 0.58092 |  0:00:02s
epoch 3  | loss: 0.52265 | val_0_auc: 0.61405 |  0:00:02s
epoch 4  | loss: 0.50749 | val_0_auc: 0.64915 |  0:00:03s
epoch 5  | loss: 0.49081 | val_0_auc: 0.68366 |  0:00:04s
epoch 6  | loss: 0.48077 | val_0_auc: 0.71573 |  0:00:05s
epoch 7  | loss: 0.4716  | val_0_auc: 0.738   |  0:00:05s
epoch 8  | loss: 0.46315 | val_0_auc: 0.75095 |  0:00:06s
epoch 9  | loss: 0.45742 | val_0_auc: 0.76472 |  0:00:07s
epoch 10 | loss: 0.45033 | val_0_auc: 0.77635 |  0:00:08s
epoch 11 | loss: 0.44597 | val_0_auc: 0.78416 |  0:00:08s
epoch 12 | loss: 0.44044 | val_0_auc: 0.79122 |  0:00:09s
epoch 13 | loss: 0.43616 | val_0_auc: 0.79897 |  0:00:10s
epoch 14 | loss: 0.43177 | val_0_auc: 0.80422 |  0:00:10s
epoch 15 | loss: 0.42849 | val_0_auc: 0.80999 |  0:00:11s
epoch 16 | loss: 0.42421 | val_0_auc: 0.81432 |  0:00:12s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
8 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/sklearn/mode

{'verbose': 1,
 'optimizer_params': {'lr': 0.02},
 'n_steps': 8,
 'n_shared': 2,
 'n_independent': 5,
 'n_a': 3,
 'momentum': 0.1,
 'lambda_sparse': 0.01,
 'gamma': 0.5,
 'clip_value': 1,
 'cat_emb_dim': [20, 20, 20, 3, 8, 20, 8, 20, 1, 5, 3, 8, 4]}

In [37]:
search.cv_results_

{'mean_fit_time': array([4.08061368e+01, 1.85347041e+02, 5.74600625e+01, 1.08301079e+02,
        2.51711654e+02, 2.32989799e+02, 5.21421972e+02, 2.84569263e-02,
        6.26589324e+02, 6.11330811e+02, 4.41757250e+01, 1.12380654e+02,
        2.24576004e+02, 5.22203013e+02, 1.21972117e+02, 4.06275187e+01,
        3.96675159e+02, 6.02129016e+01, 2.42230824e+02, 7.27539158e+01,
        1.33457010e+02, 1.24621843e+03, 1.85400095e+02, 5.61502819e+02,
        5.57822475e+01, 1.18428343e+03, 9.87606510e+02, 4.01647903e+02,
        1.14188959e+03, 5.24649794e+01, 2.75726318e-02, 4.84483572e+02,
        1.62669059e+02, 6.93285742e+01, 3.46412943e+02, 3.69374297e+02,
        8.03735483e+01, 8.09059507e+02, 5.10247952e+02, 7.30384941e+01,
        2.68254280e-02, 1.87851882e+02, 4.45718486e+01, 4.26955628e+01,
        1.39901430e+02, 3.72311754e+02, 4.55973170e+02, 1.98191815e+02,
        3.63457852e+02, 7.83383770e+02, 3.11688058e+02, 4.05035396e+02,
        3.91424727e+01, 5.72492851e+02, 5.16334

In [38]:
search.best_params_

{'verbose': 1,
 'optimizer_params': {'lr': 0.02},
 'n_steps': 8,
 'n_shared': 2,
 'n_independent': 5,
 'n_a': 3,
 'momentum': 0.1,
 'lambda_sparse': 0.01,
 'gamma': 0.5,
 'clip_value': 1,
 'cat_emb_dim': [20, 20, 20, 3, 8, 20, 8, 20, 1, 5, 3, 8, 4]}

After searching for best params, we need to retrain a model, we chose a make that on 5 folds.


In [39]:
from sklearn.model_selection import StratifiedKFold

In [40]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [41]:
models = []
for train_index, valid_index in skf.split(X, y):
    clf = TabNetClassifier(cat_idxs=cat_idxs, cat_dims=cat_dims, device_name='cpu', **search.best_params_)
    clf.fit(
        X[train_index],
        y[train_index],
        patience=20,
        # X_valid=X[valid_index],
        # y_valid=y[valid_index],
        eval_set=[(X[valid_index], y[valid_index])],
    )
    models.append(clf)

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.60107 | val_0_auc: 0.77976 |  0:00:07s
epoch 1  | loss: 0.43793 | val_0_auc: 0.84023 |  0:00:14s
epoch 2  | loss: 0.4     | val_0_auc: 0.85954 |  0:00:20s
epoch 3  | loss: 0.37987 | val_0_auc: 0.87103 |  0:00:26s
epoch 4  | loss: 0.36643 | val_0_auc: 0.88439 |  0:00:33s
epoch 5  | loss: 0.34755 | val_0_auc: 0.89597 |  0:00:41s
epoch 6  | loss: 0.33228 | val_0_auc: 0.89988 |  0:00:47s
epoch 7  | loss: 0.32411 | val_0_auc: 0.90841 |  0:00:53s
epoch 8  | loss: 0.31601 | val_0_auc: 0.91378 |  0:01:00s
epoch 9  | loss: 0.30695 | val_0_auc: 0.91423 |  0:01:06s
epoch 10 | loss: 0.3037  | val_0_auc: 0.91635 |  0:01:12s
epoch 11 | loss: 0.30079 | val_0_auc: 0.92064 |  0:01:18s
epoch 12 | loss: 0.29784 | val_0_auc: 0.91981 |  0:01:25s
epoch 13 | loss: 0.29549 | val_0_auc: 0.92228 |  0:01:31s
epoch 14 | loss: 0.29373 | val_0_auc: 0.92194 |  0:01:38s
epoch 15 | loss: 0.28967 | val_0_auc: 0.9221  |  0:01:44s
epoch 16 | loss: 0.2872  | val_0_auc: 0.92047 |  0:01:50s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.60315 | val_0_auc: 0.79925 |  0:00:05s
epoch 1  | loss: 0.45187 | val_0_auc: 0.85005 |  0:00:13s
epoch 2  | loss: 0.41627 | val_0_auc: 0.87023 |  0:00:19s
epoch 3  | loss: 0.38748 | val_0_auc: 0.88243 |  0:00:26s
epoch 4  | loss: 0.37249 | val_0_auc: 0.88876 |  0:00:32s
epoch 5  | loss: 0.3645  | val_0_auc: 0.90044 |  0:00:38s
epoch 6  | loss: 0.34853 | val_0_auc: 0.90784 |  0:00:45s
epoch 7  | loss: 0.33398 | val_0_auc: 0.91265 |  0:00:51s
epoch 8  | loss: 0.32491 | val_0_auc: 0.91579 |  0:00:58s
epoch 9  | loss: 0.31792 | val_0_auc: 0.9202  |  0:01:03s
epoch 10 | loss: 0.31434 | val_0_auc: 0.9227  |  0:01:10s
epoch 11 | loss: 0.30767 | val_0_auc: 0.92377 |  0:01:16s
epoch 12 | loss: 0.30246 | val_0_auc: 0.92575 |  0:01:22s
epoch 13 | loss: 0.30031 | val_0_auc: 0.92541 |  0:01:28s
epoch 14 | loss: 0.29791 | val_0_auc: 0.92627 |  0:01:34s
epoch 15 | loss: 0.29608 | val_0_auc: 0.92776 |  0:01:40s
epoch 16 | loss: 0.29192 | val_0_auc: 0.92696 |  0:01:46s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.58411 | val_0_auc: 0.80499 |  0:00:08s
epoch 1  | loss: 0.4322  | val_0_auc: 0.84588 |  0:00:14s
epoch 2  | loss: 0.40246 | val_0_auc: 0.86718 |  0:00:21s
epoch 3  | loss: 0.38725 | val_0_auc: 0.87852 |  0:00:28s
epoch 4  | loss: 0.36543 | val_0_auc: 0.89157 |  0:00:35s
epoch 5  | loss: 0.34746 | val_0_auc: 0.90021 |  0:00:42s
epoch 6  | loss: 0.33701 | val_0_auc: 0.90422 |  0:00:49s
epoch 7  | loss: 0.32389 | val_0_auc: 0.91324 |  0:00:55s
epoch 8  | loss: 0.31817 | val_0_auc: 0.91593 |  0:01:01s
epoch 9  | loss: 0.31084 | val_0_auc: 0.91876 |  0:01:08s
epoch 10 | loss: 0.30809 | val_0_auc: 0.92087 |  0:01:16s
epoch 11 | loss: 0.30871 | val_0_auc: 0.92232 |  0:01:22s
epoch 12 | loss: 0.30047 | val_0_auc: 0.92257 |  0:01:29s
epoch 13 | loss: 0.29914 | val_0_auc: 0.92217 |  0:01:36s
epoch 14 | loss: 0.29511 | val_0_auc: 0.92442 |  0:01:46s
epoch 15 | loss: 0.29253 | val_0_auc: 0.92438 |  0:01:55s
epoch 16 | loss: 0.29044 | val_0_auc: 0.92412 |  0:02:01s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.60508 | val_0_auc: 0.83027 |  0:00:06s
epoch 1  | loss: 0.45802 | val_0_auc: 0.86351 |  0:00:12s
epoch 2  | loss: 0.42288 | val_0_auc: 0.87235 |  0:00:18s
epoch 3  | loss: 0.39977 | val_0_auc: 0.88249 |  0:00:24s
epoch 4  | loss: 0.39076 | val_0_auc: 0.88858 |  0:00:30s
epoch 5  | loss: 0.3688  | val_0_auc: 0.89615 |  0:00:37s
epoch 6  | loss: 0.34539 | val_0_auc: 0.90579 |  0:00:43s
epoch 7  | loss: 0.32624 | val_0_auc: 0.91331 |  0:00:49s
epoch 8  | loss: 0.31671 | val_0_auc: 0.91834 |  0:00:55s
epoch 9  | loss: 0.30941 | val_0_auc: 0.91996 |  0:01:02s
epoch 10 | loss: 0.30416 | val_0_auc: 0.92375 |  0:01:08s
epoch 11 | loss: 0.30084 | val_0_auc: 0.92666 |  0:01:18s
epoch 12 | loss: 0.29745 | val_0_auc: 0.92759 |  0:01:25s
epoch 13 | loss: 0.29197 | val_0_auc: 0.92792 |  0:01:33s
epoch 14 | loss: 0.29118 | val_0_auc: 0.92816 |  0:01:42s
epoch 15 | loss: 0.28835 | val_0_auc: 0.92909 |  0:01:49s
epoch 16 | loss: 0.28657 | val_0_auc: 0.92831 |  0:01:55s
epoch 17 | los

/mnt/Dados/Mestrado_Computacao_Aplicada_UFMS/1_semestre/Inteligencia_Artificial/algoritmos-ia/venv/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [42]:
models

[TabNetClassifier(n_d=8, n_a=3, n_steps=8, gamma=0.5, cat_idxs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13], cat_dims=[120, 43, 93, 7, 17, 95, 17, 74, 3, 10, 6, 16, 8], cat_emb_dim=[20, 20, 20, 3, 8, 20, 8, 20, 1, 5, 3, 8, 4], n_independent=5, n_shared=2, epsilon=1e-15, momentum=0.1, lambda_sparse=0.01, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=14, output_dim=2, device_name='cpu', n_shared_decoder=1, n_indep_decoder=1),
 TabNetClassifier(n_d=8, n_a=3, n_steps=8, gamma=0.5, cat_idxs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13], cat_dims=[120, 43, 93, 7, 17, 95, 17, 74, 3, 10, 6, 16, 8], cat_emb_dim=[20, 20, 20, 3, 8, 20, 8, 20, 1, 5, 3, 8, 4], n_independent=5, n_shared=2, epsilon=1e-15, momentum=0.1, lambda_sparse=0.01, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, sched

In [43]:
preds = np.zeros(shape=y_test.shape)
for model in models:
    preds += clf.predict_proba(X_test)[:, 1]
preds = preds / len(models)

In [44]:
test_auc = roc_auc_score(y_score=preds, y_true=y_test)

print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

FINAL TEST SCORE FOR census-income : 0.9244184959613279
